<a href="https://colab.research.google.com/github/Jagaya/FCC-Machine-Learning/blob/main/sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  # !pip install tf-nightly
  pass
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
# !pip install tensorflow-datasets
# import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_data = pd.read_csv(
    train_file_path,
    encoding = "ISO-8859-1",
    sep="\t",
    header=0,
    names=['type', 'message'],
    usecols=['type', 'message'],
    dtype={'type': 'str', 'message': 'str'}
    )

test_data = pd.read_csv(
    test_file_path,
    encoding = "ISO-8859-1",
    sep="\t",
    header=0,
    names=['type', 'message'],
    usecols=['type', 'message'],
    dtype={'type': 'str', 'message': 'str'}
    )
train_data.head()

In [ ]:
train_data["type_code"] = train_data["type"].map({"ham": 0, "spam":1})
train_data["type"].value_counts()

In [ ]:
test_data["type_code"] = test_data["type"].map({"ham": 0, "spam":1})

In [ ]:
import re
import string
# Create a custom standardization function to strip HTML break tags '<br />'.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')

# Vocabulary size and number of words in a sequence.
vocab_size = 10000
sequence_length = 50

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = keras.layers.experimental.preprocessing.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
# text_ds = train_ds.map(lambda x, y: x)
train_array = np.array(train_data.message)

vectorize_layer.adapt(train_array)

In [ ]:
model = keras.Sequential([
                          vectorize_layer,
                          keras.layers.Embedding(
                              # Vocabulary size
                              10000,
                              # dimensions
                              32,
                              name="embedding"
                          ),
                          keras.layers.LSTM(32),
                          keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=['acc'])

history = model.fit(train_data["message"], train_data["type_code"], epochs=10, validation_split=0.2)

In [ ]:
model.predict(train_array[:1])

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred = model.predict([pred_text])[0]
  
  return [pred[0], "ham" if round(pred[0],0)==0 else "spam"]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


In [ ]:
test_cop = test_data.copy()

In [ ]:
test_cop["predict"]= model.predict(np.array(test_cop.message))
test_cop["predict"] = test_cop["predict"].round(0).astype(int)
test_cop.head()

In [ ]:
test_cop[(test_cop.type_code != test_cop.predict)]